In [86]:
import pandas as pd
import pickle
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [17]:
with open('./Data/list_actors.pickle', 'rb') as archivo:
    list_actors = pickle.load(archivo)

with open('./Data/list_directors.pickle', 'rb') as archivo:
    list_directors = pickle.load(archivo)

In [87]:
df_movie = pd.read_pickle('./Data/api.pickle')

In [131]:
df_movie.head()

,budget,popularity,release_date,revenue,title,vote_average,vote_count,release_year,return,actors,directors,release_day,release_month
0,0.0,3.860491,1988-10-21,0.0,Ariel,7.1,44.0,1988,0.000000,"[Turo Pajala, Susanna Haavisto, Matti Pellonpä...",[Aki Kaurismäki],Friday,10
1,0.0,2.292110,1986-10-16,0.0,Shadows in Paradise,7.1,35.0,1986,0.000000,"[Matti Pellonpää, Kati Outinen, Sakari Kuosman...",[Aki Kaurismäki],Thursday,10
2,4000000.0,9.026586,1995-12-09,4300000.0,Four Rooms,6.5,539.0,1995,1.075000,"[Tim Roth, Antonio Banderas, Jennifer Beals, M...","[Allison Anders, Alexandre Rockwell, Robert Ro...",Saturday,12
3,0.0,5.538671,1993-10-15,12136938.0,Judgment Night,6.4,79.0,1993,0.000000,"[Emilio Estevez, Cuba Gooding Jr., Denis Leary...",[Stephen Hopkins],Friday,10
4,11000000.0,42.149697,1977-05-25,775398007.0,Star Wars,8.1,6778.0,1977,70.490728,"[Mark Hamill, Harrison Ford, Carrie Fisher, Pe...",[George Lucas],Wednesday,5


In [89]:
df_movie.dtypes

budget                  float64
popularity              float64
release_date     datetime64[ns]
revenue                 float64
title                    object
vote_average            float64
vote_count              float64
release_year              int64
return                  float64
actors                   object
directors                object
release_day              object
release_month             int64
dtype: object

In [6]:
df_movie.isna().sum()

budget              0
popularity          0
release_date        0
revenue             0
status             80
title               0
vote_average        0
vote_count          0
release_year        0
return              0
actors           2354
directors         723
release_day         0
release_month       0
dtype: int64

In [7]:
df_movie[df_movie['actors'].isna()]

,budget,popularity,release_date,revenue,status,title,vote_average,vote_count,release_year,return,actors,directors,release_day,release_month
597,0.0,0.693917,1895-06-10,0.0,Released,Workers Leaving the Lumière Factory,6.2,52.0,1895,0.0,None,[Louis Lumière],Monday,6
1767,0.0,2.032918,1969-01-01,0.0,Released,Bambi Meets Godzilla,6.5,40.0,1969,0.0,None,[Marv Newland],Wednesday,1
2970,0.0,0.106981,2008-01-01,0.0,Released,Uncounted,9.0,1.0,2008,0.0,None,[David Earnhardt],Tuesday,1
3025,0.0,0.110946,2009-09-17,0.0,Released,It's Hard Being Loved by Jerks,7.3,3.0,2009,0.0,None,[Daniel LeConte],Thursday,9
3792,0.0,0.281609,1993-12-25,0.0,Released,La estrategia del caracol,7.2,9.0,1993,0.0,None,[Sergio Cabrera],Saturday,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45433,0.0,0.010588,2013-04-23,0.0,Released,The Everyday,0.0,0.0,2013,0.0,None,[],Tuesday,4
45434,0.0,0.249393,2017-06-13,0.0,Released,Mutafukaz,7.5,2.0,2017,0.0,None,"[Shoujirou Nishimi, Guillaume Renard]",Tuesday,6
45438,0.0,0.183256,2017-06-23,0.0,Released,Edith Walks,0.0,0.0,2017,0.0,None,[Andrew Kötting],Friday,6
45445,0.0,0.000001,1992-01-01,0.0,Released,The Making of 'The Terminator': A Retrospective,0.0,0.0,1992,0.0,None,[Jeff McQueen],Wednesday,1


In [132]:
def cantidad_filmaciones_mes(mes:str):
    months = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12
    }

    if not isinstance(mes, str):
        return 'Debe ingresar un texto, por ejemplo, por ejemplo: enero'
    elif mes.lower() not in months:
        return 'Debe ingresar un mes válido del año, por ejemplo: enero'
    else:
        mes = mes.lower()
        month = months[mes]
        resp = df_movie[(df_movie['release_month'] == month)]['release_month'].count()
        return {'mes': mes, 'cantidad': str(resp)}

In [133]:
months = {
        "enero": 1,
        "febrero": 2,
        "marzo": 3,
        "abril": 4,
        "mayo": 5,
        "junio": 6,
        "julio": 7,
        "agosto": 8,
        "septiembre": 9,
        "octubre": 10,
        "noviembre": 11,
        "diciembre": 12,
        "sabado":0,
        12:0
    }
for a in months:
    print(cantidad_filmaciones_mes(a))

{'mes': 'enero', 'cantidad': '5831'}
{'mes': 'febrero', 'cantidad': '3006'}
{'mes': 'marzo', 'cantidad': '3532'}
{'mes': 'abril', 'cantidad': '3423'}
{'mes': 'mayo', 'cantidad': '3325'}
{'mes': 'junio', 'cantidad': '3131'}
{'mes': 'julio', 'cantidad': '2630'}
{'mes': 'agosto', 'cantidad': '3357'}
{'mes': 'septiembre', 'cantidad': '4798'}
{'mes': 'octubre', 'cantidad': '4581'}
{'mes': 'noviembre', 'cantidad': '3629'}
{'mes': 'diciembre', 'cantidad': '3766'}
Debe ingresar un mes válido del año, por ejemplo: enero
Debe ingresar un texto, por ejemplo, por ejemplo: enero


In [98]:
def cantidad_filmaciones_dia(dia:str):
    days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miércoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sábado': 'Saturday',
    'domingo': 'Sunday'}

    if not isinstance(dia, str):
        return 'Debe ingresar un texto, por ejemplo: sabado'
    elif dia.lower() not in days:
        return 'Debe ingresar un dia de la semana, por ejemplo: sabado'
    else:
        dia = dia.lower()
        day = days[dia]
        resp = df_movie[(df_movie['release_day'] == day)]['release_day'].count()
        return {'dia':dia, 'cantidad':resp}

In [99]:
days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miércoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sábado': 'Saturday',
    'domingo': 'Sunday',
    "enero": 0,
    12:0}
for a in days:
    print(cantidad_filmaciones_dia(a))

{'dia': 'lunes', 'cantidad': 3474}
{'dia': 'martes', 'cantidad': 4598}
{'dia': 'miércoles', 'cantidad': 6986}
{'dia': 'jueves', 'cantidad': 7466}
{'dia': 'viernes', 'cantidad': 13783}
{'dia': 'sábado', 'cantidad': 5112}
{'dia': 'domingo', 'cantidad': 3590}
Debe ingresar un dia de la semana, por ejemplo: sabado
Debe ingresar un texto, por ejemplo: sabado


In [129]:
def score_titulo(titulo:str):
    if not isinstance(titulo, str):
        return 'Debe ingresar un texto'
    elif not (df_movie == titulo).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    else:
        resp = df_movie[df_movie['title'] == titulo][['title','popularity','release_year']].to_dict(orient='records')[0]
        return {'titulo':resp['title'], 'popularidad':resp['popularity'], 'año':resp['release_year']}

In [130]:
print(score_titulo('123'))
print(score_titulo('jumanji'))
print(score_titulo(6534))

La pelicula no se encuentra en la base de datos
La pelicula no se encuentra en la base de datos
Debe ingresar un texto


In [102]:
def votos_titulo(titulo:str):
    if not isinstance(titulo, str):
        return 'Debe ingresar un texto'
    elif not (df_movie == titulo).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    elif int(df_movie[df_movie['title'] == titulo]['vote_count']) > 2000:
        resp = df_movie[df_movie['title'] == titulo][['title','vote_count','vote_average']].to_dict(orient='records')[0]
        return {'titulo':resp['title'], 'voto_total':resp['vote_count'], 'voto_promedio':resp['vote_average']}
    else:
        return 'La consulta no cumple con las condicion por ende no se devuelve informacion'

In [103]:
votos_titulo('sgw54')

'La pelicula no se encuentra en la base de datos'

In [112]:
def get_actor(nombre_actor:str):
    if not isinstance(nombre_actor, str):
        return 'Debe ingresar un texto'
    
    elif not any(nombre_actor in sublist for sublist in df_movie['actors'] if sublist):
        return 'El actor no se encuentra en la base de datos'

    _return = 0
    n_films = 0
    avg = 0
    for index,lista in enumerate(df_movie['actors']):
        if lista: 
            if nombre_actor in lista:
                n_films += 1
                _return += df_movie['return'].iloc[index]
    
    if _return == 0 and n_films == 0:
        avg = 0
    else:
        avg = _return/n_films

    return {'actor':nombre_actor, 'cantidad_filmaciones':n_films, 'retorno_total':_return, 'retorno_promedio':avg}

In [113]:
get_actor('sdfsd')

'El actor no se encuentra en la base de datos'

In [125]:
def get_director(nombre_director:str):
    if not isinstance(nombre_director, str):
        return 'Debe ingresar un texto'
    elif not any(nombre_director in sublist for sublist in df_movie['directors'] if sublist):
        return 'El director no se encuentra en la base de datos'

    total_return = 0
    movies = []
    years = []
    _return = []
    budget = []
    revenue = []
    for index,lista in enumerate(df_movie['directors']):
        if lista:
            if nombre_director in lista:
                total_return += df_movie['return'][index]
                movies.append(df_movie['title'][index])
                years.append(df_movie['release_year'][index])
                _return.append(df_movie['return'][index])
                budget.append(df_movie['budget'][index])
                revenue.append(df_movie['revenue'][index])

    return {'director':nombre_director, 'retorno_total_director':total_return, 
    'peliculas':movies, 'año':years, 'retorno_pelicula':_return, 
    'budget_pelicula':budget, 'revenue_pelicula':revenue}

In [126]:
get_director('Aki Kaurismäki')

{'director': 'Aki Kaurismäki',
 'retorno_total_director': 3.079139631884058,
 'peliculas': ['Ariel',
  'Shadows in Paradise',
  'Lights in the Dusk',
  'The Man Without a Past',
  'The Match Factory Girl',
  'I Hired a Contract Killer',
  'Drifting Clouds',
  'Hamlet Goes Business',
  'Visions of Europe',
  'Leningrad Cowboys Go America',
  'Ten Minutes Older: The Trumpet',
  'Chloe',
  'An Unreasonable Man',
  'More',
  'Blood Out',
  'The Mask',
  "Saimir's decision",
  'Mr. & Mrs. Incredible',
  'Safe',
  'Hansel and Gretel',
  'Fire-Eater',
  'The Wait',
  'Running Wild',
  'Justin Timberlake + The Tennessee Kids'],
 'año': [1988,
  1986,
  2006,
  2002,
  1990,
  1990,
  1996,
  1987,
  2004,
  1989,
  2002,
  2009,
  2007,
  1998,
  2011,
  1961,
  2004,
  2011,
  2012,
  1988,
  1998,
  2013,
  1995,
  2016],
 'retorno_pelicula': [0.0,
  0.0,
  1.1703028985507247,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.5639638666666666,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0

In [127]:
def recomendacion(titulo:str):
    if not isinstance(titulo, str):
        return 'Debe ingresar un texto'
    elif not (df_movie == titulo).any().any():
        return 'La pelicula no se encuentra en la base de datos'
    # Cargo los modelos entrenados
    tfidf_model = joblib.load("./Models/tfidf_model.pkl")
    knn_model = joblib.load("./Models/knn_model.pkl")

    title_vector = tfidf_model.transform([titulo])  # Transforma el título de entrada en una representación vectorial
    _, indices = knn_model.kneighbors(title_vector)  # Busca los vecinos más cercanos basados en la representación vectorial
    recommendations = df_movie["title"].iloc[indices[0][1:]].tolist()  # Obtiene los títulos de las películas más similares (excluyendo la película de referencia)
    return {'lista recomendada': str(recommendations)}

In [128]:
recomendacion("Manuel on the Island of Wonders")

{'lista recomendada': "['Un marido de ida y vuelta', 'The Sandwich Man', 'Chernobyl Diaries', 'Friday the 13th: A New Beginning', 'Aftershock']"}